In [3]:
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import streamlit as st
from streamlit_chat import message
import sys
from PyQt5.QtWidgets import QApplication
from telepot.loop import MessageLoop
from dotenv import load_dotenv
import telepot
load_dotenv(verbose=True)
import json
from konlpy.tag import Okt
okt=Okt()

bot_number'       '

2022-12-06 23:52:12.265 INFO    dotenv.main: Python-dotenv could not find configuration file .env.


### 데이터 로드

In [4]:
df=pd.read_csv('mv.csv',header=None)

In [5]:
df.head()

,0,1
0,와일드 로즈,('고만고만한 음악 영화가 아니다. 뮤지션의 꿈을 가진 주인공이 역경을 딛고 일어서...
1,해치지않아,"('자연과 인간의 공존, 무분별한 개발에 대한 경고, 비정규직과 고용 불안 문제 등..."
2,임파서블 러브,"('전반부가 사랑 이야기라면, 후반부는 그 사랑이 할퀴고 간 쓰린 고통과 마주한 모..."
3,어린 의뢰인,('영화의 메시지는 강하게 와닿는다. 서두에 ‘제노비스 살인 사건’을 언급하며 목격...
4,기방도령,"('소재만 솔깃하다. 소재를 발전시켜 나가는 아이디어는 조악하고, 갈등을 짜나가는 ..."


크롤링한 데이터를 불러온다.

### 전처리

In [6]:
def export_noun(df):
    for i in range(len(df)):
        try:
            df.iloc[i][1]=os.path.join(*okt.nouns(df.iloc[i][1])).replace("/"," ")
        except:
            pass

export_noun(df)

In [7]:
df.head()

,0,1
0,와일드 로즈,음악 영화 뮤지션 꿈 주인공 역경 스타 탄생 영화 자신 목소리 이야기 스코틀랜드 글...
1,해치지않아,자연 인간 공존 무분별 개발 대한 경고 비정규직 고용 불안 문제 등 사회 주제 소재...
2,임파서블 러브,전반 부가 사랑 이야기 라면 후반 그 사랑 간 고통 마주 모녀 이야기 때 사랑 영화...
3,어린 의뢰인,영화 메시지 서두 노비 스 살인 사건 언급 목격자 유 무죄 영화 아동학대 또한 다수...
4,기방도령,소재 소재 발전 아이디어 갈등 방식 병맛 승부 것 급 코미디 로 즐 기기 로맨스 전...


형태소가 명사인 단어만 추출하여 전처리를 해주었다.

형태소 분석기는 Open Korean Text를 사용하였다.

In [8]:
model = TfidfVectorizer()
train_model=model.fit_transform(df[1])

문장 내에서의 단어의 빈도수와 전체 텍스트에서 단어의 빈도수를 고려하는 TfidfVectorizer를 사용하여 영화의 특징을 담고 있는 단어에 가중치를 부여한다.

### 영화 추천

In [9]:
def get_output(a):
    li=okt.nouns(a)                              #사용자로부터 입력받은 텍스트를 전처리
    a=[os.path.join(*li).replace("/"," ")]
    a=model.transform(a)                         #위에서 훈련한 모델로 임베딩
    sim1 = cosine_similarity(train_model,a)      #입력받은 텍스트와 위에서 수치화한 영화 리뷰간의 코사인 유사도 계산
    if len(li)>2:                                #입력된 텍스트의 단어 개수가 3개 이상일 경우,     
        mv=df.iloc[np.where(sim1==max(sim1))][0] #코사인 유사도가 가장 높은 영화 추천
        mv=str(mv).split('\n')[0][5:].strip()
    else :                                        
        mv=df.iloc[random.choice(np.where(sim1>max(sim1)-0.15)[0])][0]
    return mv

코사인 유사도를 이용하여 사용자로 입력받은 문장과 비교하여 유사도가 가장 높은 영화를 추천한다.

다만, 사용한 입력한 텍스트가 문장이 아닌 단순 키워드(ex. '공포' 와 같이 영화 장르나 '사랑'과 같은 포괄적인 주제)일 경우 코사인 유사도가 가장 높은 영화 하나만을 추천하는 것 보다는 관련된 영화를 다양하게 추천하는 것이 낫다고 생각하여 가장 높은 코사인 유사도에서 0.15를 뺀 값 이상의 유사도를 가진 영화를 랜덤하게 추천하는 방식으로 하였다.

In [10]:
a='전투가 치열한 전쟁 영화'
get_output(a)

'1942: 언노운 배틀'

In [11]:
for i in range(5):
    print(get_output('전쟁'))

1942: 언노운 배틀
전장의 피아니스트
1942: 언노운 배틀
폴란드로 간 아이들
폴란드로 간 아이들


'전쟁'이라는 키워드만 입력했을 경우 시행할 때 마다 다른 영화가 추천되는 것을 볼 수 있다.

### 챗봇에 연결

In [ ]:
def handle(msg):
    content_type, chat_type, chat_id, msg_date, msg_id=telepot.glance(msg, long=True)
    
    first_msg = '안녕하세요! 저는 여러분의 영화 추천 챗봇 "무비 메이트"예요.\n 보고싶은 영화의 내용을 적어주면 영화를 추천할게요.'
    output_msg = '이 영화를 추천할게요.\n\n'
    
    if content_type !='text':
        bot.sendMessage('제가 이해할 수 없는 말이에요.')
    print(msg)
    
    if msg['text']=='/start':
        bot.sendMessage(chat_id, first_msg)
    else:
        mv=get_output(msg['text'])
        bot.sendMessage(chat_id, output_msg+'<'+str(mv)+'>')
        
        
bot=telepot.Bot(bot_number)
MessageLoop(bot, handle).run_forever()

{'message_id': 283, 'from': {'id': 5506263738, 'is_bot': False, 'first_name': 'Ys', 'last_name': 'Park', 'language_code': 'ko'}, 'chat': {'id': 5506263738, 'first_name': 'Ys', 'last_name': 'Park', 'type': 'private'}, 'date': 1670338054, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}
{'message_id': 285, 'from': {'id': 5506263738, 'is_bot': False, 'first_name': 'Ys', 'last_name': 'Park', 'language_code': 'ko'}, 'chat': {'id': 5506263738, 'first_name': 'Ys', 'last_name': 'Park', 'type': 'private'}, 'date': 1670338087, 'text': '스포츠 영화 추천해줘'}
{'message_id': 287, 'from': {'id': 5506263738, 'is_bot': False, 'first_name': 'Ys', 'last_name': 'Park', 'language_code': 'ko'}, 'chat': {'id': 5506263738, 'first_name': 'Ys', 'last_name': 'Park', 'type': 'private'}, 'date': 1670338105, 'text': '이별도 하고 다시 만나는 절절한 사랑 이야기'}
{'message_id': 289, 'from': {'id': 5506263738, 'is_bot': False, 'first_name': 'Ys', 'last_name': 'Park', 'language_code': 'ko'}, 'chat': {'id': 550

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/telepot/loop.py", line 36, in run_forever
    msg = self._inqueue.get(block=True)
  File "/opt/anaconda3/lib/python3.9/queue.py", line 171, in get
    self.not_empty.wait()
  File "/opt/anaconda3/lib/python3.9/threading.py", line 312, in wait
    waiter.acquire()
KeyboardInterrupt


In [ ]:
# id=